<a href="https://colab.research.google.com/github/GunH-colab/CompVis/blob/main/facial_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import sys
import random
import subprocess
import cv2

import numpy as np
import pandas as pd

#We are using keras as our base package to build our classifier. 
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image
from keras.utils.vis_utils import plot_model
from keras.callbacks import  EarlyStopping

import gc
import matplotlib.pyplot as plt

import tensorflow as tf
from tqdm import tqdm

'''
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
'''

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [2]:
def run_cmd(cmd, stderr = subprocess.STDOUT):
    out = None
    try:
        out = subprocess.check_output([cmd], shell=True, stderr = subprocess.STDOUT, universal_newlines = True)
    except subprocess.CalledProcessError as e:
        print(f'ERROR {e.returncode}: {cmd}\n\t{e.output}', flush=True, file=sys.stderr)
        raise e
    return out

def clone_data(data_root):
    clone_uri = 'https://github.com/muxspace/facial_expressions'
    if os.path.exists(data_root):
        assert os.path.isdir(data_root), \
        f'{data_root} should be cloned from {clone_uri}'
    else:
        print('Cloning the covid facial expression data dataset. It may take a while\n...\n', flush=True)
        run_cmd(f'git clone {clone_uri} {data_root}')

In [3]:
data_root = "./data"

In [4]:
clone_data(data_root)

Cloning the covid facial expression data dataset. It may take a while
...



In [5]:
labels = pd.read_csv('/content/data/data/legend.csv')
labels['target'] = labels.emotion.str.lower()
labels.drop('emotion', axis = 1, inplace = True)

labels.target.value_counts()

neutral      6868
happiness    5696
surprise      368
sadness       268
anger         252
disgust       208
fear           21
contempt        9
Name: target, dtype: int64

In [6]:
import shutil

labels = labels.sort_values('target')
class_names = list(labels.target.unique())

train_images = '/content/data/images'
train_cat = '/train_'

for i in class_names:
    os.makedirs(os.path.join('train_', i))

for c in class_names: # Category Name
    for i in list(labels[labels['target']==c]['image']): # Image Id
        get_image = os.path.join('/content/data/images', i) # Path to Images
        move_image_to_cat = shutil.copy(get_image, 'train_/'+c)

In [7]:
def img_gen(train_path_variable):
  train_preprocess = keras.preprocessing.image.ImageDataGenerator(
                                        rescale = 1./255,
                                        zoom_range = 0.2,
                                        shear_range = 0.2,
                                        horizontal_flip = True,
                                        validation_split = 0.2,
                                    )

  train = train_preprocess.flow_from_directory(
                                        train_path_variable,
                                        target_size = (224, 224),
                                        batch_size = 16,
                                        class_mode = 'categorical',
                                        subset = 'training'
                                    )
  val = train_preprocess.flow_from_directory(
                                        train_path_variable,
                                        target_size = (224, 224),
                                        batch_size = 16,
                                        class_mode = 'categorical',
                                        subset = 'validation'
                                    )
  return train, val

In [8]:
train, val = img_gen('/content/train_')

Found 10951 images belonging to 8 classes.
Found 2732 images belonging to 8 classes.


In [9]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation = 'relu', input_shape = (224, 224, 3)))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(8, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
early_stop = EarlyStopping(monitor='val_loss', patience = 10)

In [10]:
train.class_indices

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'happiness': 4,
 'neutral': 5,
 'sadness': 6,
 'surprise': 7}

In [11]:
history = model.fit(
                                train,
                                validation_data = val,
                                epochs = 50,
                                callbacks = [early_stop],
                                batch_size = 16
                    )

Epoch 1/50
685/685 [==============================] - 230s 291ms/step - loss: 1.1753 - accuracy: 0.4630 - val_loss: 0.9731 - val_accuracy: 0.5015
Epoch 2/50
685/685 [==============================] - 197s 288ms/step - loss: 1.0256 - accuracy: 0.5072 - val_loss: 0.9648 - val_accuracy: 0.5128
Epoch 3/50
685/685 [==============================] - 196s 286ms/step - loss: 0.9878 - accuracy: 0.5510 - val_loss: 0.8475 - val_accuracy: 0.6680
Epoch 4/50
685/685 [==============================] - 197s 288ms/step - loss: 0.9202 - accuracy: 0.6334 - val_loss: 0.7974 - val_accuracy: 0.7160
Epoch 5/50
685/685 [==============================] - 201s 293ms/step - loss: 0.8814 - accuracy: 0.6655 - val_loss: 0.7623 - val_accuracy: 0.7189
Epoch 6/50
685/685 [==============================] - 200s 292ms/step - loss: 0.8300 - accuracy: 0.7002 - val_loss: 0.7400 - val_accuracy: 0.7233
Epoch 7/50
685/685 [==============================] - 201s 293ms/step - loss: 0.8203 - accuracy: 0.7040 - val_loss: 0.6688 -